In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
gc.enable()

In [ ]:
PLANET_KAGGLE_ROOT = os.path.abspath("../input/")
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, '../input/planets-dataset/planet/planet/train-jpg/')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, '../input/planets-dataset/planet/planet/train_classes.csv')

In [ ]:
train_classes = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)

In [ ]:
labels = set()
for i in range(len(train_classes)):
    tags = train_classes['tags'][i].split(' ')
    labels.update(tags)
    
    
label_list = list(labels)
label_list

In [ ]:
# Add onehot features for every label
for label in label_list:
    train_classes[label] = train_classes['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)

train_classes["image_name"] = train_classes["image_name"] + ".jpg"

train_classes.head()

In [ ]:
y_col = list(train_classes.columns[2:])

In [ ]:
import tensorflow as tf
from sklearn.metrics import fbeta_score
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras import backend
from keras.models import Model
from keras.layers import Dropout

In [ ]:
# initializing an image generator with some data augumentation
image_gen = ImageDataGenerator(rescale=1./255.)

# loading images from dataframe
X = image_gen.flow_from_dataframe(dataframe=train_classes,
        directory='/kaggle/input/planets-dataset/planet/planet/train-jpg/', x_col='image_name', y_col=y_col,
       target_size=(128, 128), class_mode='raw', seed=1, batch_size=128)

In [ ]:
def fbeta(y_true , y_pred, beta=2, epsilon=1e-4):
    squared_beta = beta**2

    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.2)), tf.float32)
        
    tp = tf.reduce_sum(y_true * y_pred, axis=1)
    fp = tf.reduce_sum(y_pred, axis=1) - tp
    fn = tf.reduce_sum(y_true, axis=1) - tp
    
    p = tp/(tp+fp+epsilon)   #precision
    r = tp/(tp+fn+epsilon)   #recall
    
    fb = (1+squared_beta)*p*r / (squared_beta*p + r + epsilon)
    return fb

In [ ]:
train_image_generator =ImageDataGenerator(rescale=1/255,validation_split=0.1,
                                          rotation_range = 180, horizontal_flip = True)

# using a validation split here

# generating the 90% training image data
train_generator = train_image_generator.flow_from_dataframe(dataframe=train_classes,
        directory='../input/planets-dataset/planet/planet/train-jpg/', x_col='image_name', y_col=y_col,
       target_size=(128, 128), class_mode='raw', seed=0, batch_size=128, subset='training')

# generating the 10% validation image data
validation_generator = train_image_generator.flow_from_dataframe(dataframe=train_classes,
        directory='../input/planets-dataset/planet/planet/train-jpg/', x_col='image_name', y_col=y_col,
       target_size=(128, 128), class_mode='raw', seed=0, batch_size=128, subset='validation')

In [ ]:
# setting step size for training and validation image data
train_step_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
val_step_size = int(np.ceil(validation_generator.samples / train_generator.batch_size))

In [ ]:
# from tensorflow import keras

# Using Resnet50 model

in_shape=(128, 128, 3)
out_shape=17


resnet50 = keras.applications.resnet50
conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=in_shape)

# adding new classifier layers
layer1 = Flatten()(conv_model.layers[-1].output)
layer2 = Dense(128, activation='relu', kernel_initializer='he_uniform')(layer1)
layer3 = Dense(128, activation='relu', kernel_initializer='he_uniform')(layer2)
layer4 = Dense(128, activation='relu', kernel_initializer='he_uniform')(layer3)
predictions = Dense(out_shape, activation='sigmoid')(class2)

full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

In [ ]:
# compile model
opt = keras.optimizers.Adam(learning_rate=0.0001)
full_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint(filepath='resnet50_model_wts.hdf5',monitor='val_fbeta', 
                             save_best_only=True, save_weights_only=True, mode='max', verbose = 1)
early = EarlyStopping(monitor='val_fbeta', min_delta=0, patience=10, verbose=1, mode='max')

In [ ]:
# fit model
history = full_model.fit(train_generator, steps_per_epoch=train_step_size, 
                                  validation_data=validation_generator, validation_steps=val_step_size, 
                                   epochs=30, callbacks=[checkpoint,early])

In [ ]:
plt.figure(1, figsize = (8,8)) 
plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='test')
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 

# plot accuracy
plt.subplot(212)
plt.title('Fbeta')
plt.plot(history.history['fbeta'], color='blue', label='train')
plt.plot(history.history['val_fbeta'], color='orange', label='test')
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 

In [ ]:
model2 = full_model # building a sequential model for testing

#loading in the weights of the trained model
model2.load_weights('resnet50_model_wts.hdf5')

In [ ]:
sample_submission_df = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
sample_submission_df.head()

# adding .jpg extension to 'image_name' in sample_submission data
sample_sub = sample_submission_df.copy()
sample_sub['image_name'] = sample_sub['image_name'].apply(lambda i: '{}.jpg'.format(i))
sample_sub.head()

In [ ]:
# selecting the first 40669 image_name from the sample_submission to generate image data for test.jpg
test1 = sample_sub.iloc[:40669]['image_name'].reset_index().drop('index', axis=1)
test1.head()

In [ ]:
# to check the shape of the first test
test1.shape

In [ ]:
# initializing an image data generator object for the test1
test_image_generator1 = ImageDataGenerator(rescale=1/255)


test_generator1 = test_image_generator1.flow_from_dataframe(dataframe=test1,
            directory='../input/planets-dataset/planet/planet/test-jpg', x_col='image_name', y_col=None,
            batch_size=128, shuffle=False, class_mode=None, target_size=(128, 128))

# setting the step size 
test_step_size1 = int(np.ceil(test_generator1.samples / test_generator1.batch_size))

In [ ]:
# reseting the generator to avoid shuffling

test_generator1.reset() 

# prediction for test1
prediction1 = model2.predict(test_generator1, steps=test_step_size1, verbose=1) 

In [ ]:
# obtaining the filenames from test_generator
filenames1 = test_generator1.filenames 
        
# converting the predictions of the first 40669 to tag names
predict_tags1 = pd.DataFrame(prediction1)
predict_tags1 = predict_tags1.apply(lambda x: ' '.join(np.array(label_list)[x > 0.2]), axis=1)

# converting the predictions of the first 40669 to a dataframe
result_df1 = pd.DataFrame({'image_name': filenames1, 'tags': predict_tags1})
result_df1.head()

In [ ]:
# selecting the remaining image_name from the sample_submission to generate image data for test-additional.jpg
test2 = sample_sub.iloc[40669:]['image_name'].reset_index().drop('index', axis=1)
test2.head()

In [ ]:
# initializing an image data generator object for the remaining images in the sample submission dataframe
test_image_generator2 = ImageDataGenerator(rescale=1/255)


test_generator2 = test_image_generator2.flow_from_dataframe(dataframe=test2, 
            directory='../input/planets-dataset/test-jpg-additional/test-jpg-additional', x_col='image_name', 
            y_col=None, batch_size=128, shuffle=False, class_mode=None, target_size=(128, 128))

# setting the step size for test2
test_step_size2 = int(np.ceil(test_generator2.samples / test_generator2.batch_size))

In [ ]:
# reseting the generator to avoid shuffling
test_generator2.reset() 

# prediction for test2
prediction2 = model2.predict(test_generator2, steps=test_step_size2, verbose=1)

In [ ]:
# obtaining the filenames from test_generator2
filenames2 = test_generator2.filenames 
        
# converting the predictions of the remaining images to tag names
predict_tags2 = pd.DataFrame(prediction2)
predict_tags2 = predict_tags2.apply(lambda x: ' '.join(np.array(label_list)[x > 0.2]), axis=1)

# converting the predictions of the remaining to a dataframe
result_df2 = pd.DataFrame({'image_name': filenames2, 'tags': predict_tags2})

In [ ]:
final_result = pd.concat([result_df1, result_df2]) 
    
final_result = final_result.reset_index().drop('index', axis=1)

print(final_result.shape)
final_result.head()

In [ ]:
assert sum(sample_sub['image_name'] == final_result['image_name']) == 61191

In [ ]:
final_result['image_name'] = final_result['image_name'].apply(lambda x: x[:-4])
final_result.head()

In [ ]:
final_result.to_csv('best_submission.csv', index=False)